# **Interactive Visual Analytics with Folium**

In this notebook, I will work with the SpaceX dataset to analyze launch sites and calculate distances to key infrastructure, such as cities, railways, highways, and coastlines, using the Folium mapping library in Python. This interactive visualization will allow me to explore spatial relationships and enhance understanding of SpaceX launch site locations.

Let's first import the required Python packages

In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium 
import pandas as pd

In [3]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

We will use the spacex_launch_geo.csv dataset, which is an augmented dataset with latitude and longitude added for each site.

In [4]:
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df = pd.read_csv(spacex_csv_file)

Now, let's take a look at what are the coordinates for each site.

In [5]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [6]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

Let's add highlightened circle area with a text label on each coordinate using folium.Circle

In [7]:
circle = folium.Circle(
    nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(
    folium.Popup('NASA Johnson Space Center'
    ))
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )

)

site_map.add_child(circle)
site_map.add_child(marker)

Now, let's add a circle for each launch site in data frame launch_sites with a for loop


In [8]:
from IPython.display import display

mymap = folium.Map(location= [launch_sites_df.iloc[0]['Lat'], launch_sites_df.iloc[0]['Long']], zoom_start=5)

#Loop to get each launch site
for _, row in launch_sites_df.iterrows():
    coordinate= [row['Lat'], row['Long']]
    folium.Circle(
        location=coordinate,
        radius=1000,
        color='black',
        fill=True,
        fill_color='yellow'
    ).add_child(folium.Popup(row['Launch Site'])).add_to(mymap)

    folium.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{row["Launch Site"]}</b></div>'
        )
    ).add_to(mymap)
display(mymap)


Next, let's create markers for all launch records. If a launch was successful (class=1), then we use a green marker and if a launch was failed, we use a red marker (class=0)

In [9]:
# Define marker colors based on launch success/failure

spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x==1 else 'red')

# Create a Folium map centered around the first launch site

mymap = folium.Map(location=[spacex_df.iloc[0]['Lat'], spacex_df.iloc[0]['Long']], zoom_start=10)
marker_cluster = MarkerCluster().add_to(mymap)

#Loop through each launch site and add markers and a cicle
for index, row in spacex_df.iterrows():
    coordinate= [row['Lat'], row['Long']]
    folium.Circle(
        location=coordinate,
        radius= 1000,
        color='black',
        fill_color= 'yellow',
        fill=True
        
    ).add_child(folium.Popup(row['Launch Site'])).add_to(mymap)


    folium.Marker(
        location= coordinate,
        icon= DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html= f'<div style ="font size:12px; color:#d35400;"> <b>{row['Launch Site']}</b></div>'
            
        )
    ).add_to(mymap)

    marker= folium.Marker(
        location=coordinate,
        popup= f'{row['Launch Site']} - {'Success' if ['class'] == 1 else 'Failed'}',
        icon=folium.Icon(color='white', icon_color=row['marker_color'])
    )
    marker_cluster.add_child(marker)
mymap




Next, we need to explore and analyze the proximities of launch sites.

In [10]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over the  map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator='Long',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat',
    lat_formatter=formatter,
    long_formatter= formatter,
)

mymap.add_child(mouse_position)
mymap

A function that will calculate distance:

In [11]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1,lon1,lat2,lon2):
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

Let's mark down a point on the closest coastline using MousePosition and calculate the distance between the closest **coastline point** and the **launch site KSC LC-39A**	.

In [13]:
# launch site coordinates 
launch_site_lat = 28.5733
launch_site_lon = -80.6513

# Record of the closest coastline coordinates 
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance between the launch site and the coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a Folium map centered on the launch site
mymap = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Enable MousePosition plugin to track coordinates
MousePosition(
    position='topright',
    separator=' | ',
    empty_string='No coordinates',
    lng_first=False,
    num_digits=6
).add_to(mymap)

# Add markers for launch site and coastline
folium.Marker([launch_site_lat, launch_site_lon], popup="Launch Site", icon=folium.Icon(color='blue')).add_to(mymap)
folium.Marker([coastline_lat, coastline_lon], popup="Closest Coastline", icon=folium.Icon(color='green')).add_to(mymap)

# Add a marker showing distance to coastline using DivIcon
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)
distance_marker.add_to(mymap)

folium.PolyLine([(launch_site_lat, launch_site_lon), (coastline_lat, coastline_lon)], color='red', weight=1).add_to(mymap)

# display map
mymap


### Similarly, I will draw a line between the Kennedy Space Center to its closest city, railway, and highway.

The closest city to the **KSC LC-39A launch site** is Titusville, Florida. It's located near the Kennedy Space Center, making it a key area for spaceX activities.

In [96]:
# launch site coordinates 
launch_site_lat = 28.5733
launch_site_lon = -80.6513

# Record of Titusville, Florida
city_lat = 28.6122
city_lon = -80.8076

# Calculate the distance between the launch site and the coastline
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Create a Folium map centered on the launch site
mymap = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Enable MousePosition plugin to track coordinates
MousePosition(
    position='topright',
    separator=' | ',
    empty_string='No coordinates',
    lng_first=False,
    num_digits=6
).add_to(mymap)

# Add markers for launch site and coastline
folium.Marker([launch_site_lat, launch_site_lon], popup="Launch Site", icon=folium.Icon(color='blue')).add_to(mymap)
folium.Marker([city_lat, city_lon], popup=" Titusville", icon=folium.Icon(color='green')).add_to(mymap)

# Add a marker showing distance to coastline using DivIcon
distance_marker = folium.Marker(
    location=[city_lat, city_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
    )
)
distance_marker.add_to(mymap)

folium.PolyLine([(launch_site_lat, launch_site_lon), (city_lat, city_lon)], color='red').add_to(mymap)

# display map
mymap


According to the website tri-rail.com, the closest railway station to KSC LC-39A launch site (Kennedy Space Center) is Tri-Rail Opa-locka Station, located at 480 Ali Baba Ave, Opa-Locka, FL, 33054.

In [97]:
# launch site coordinates 
launch_site_lat = 28.5733
launch_site_lon = -80.6513

# Record of closest railway (Tri-Rail Opa-locka Station) 
rail_lat =  25.9003
rail_lon = -80.2534

# Calculate the distance between the launch site and the coastline
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, rail_lat, rail_lon)

# Create a Folium map centered on the launch site
mymap = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Enable MousePosition plugin to track coordinates
MousePosition(
    position='topright',
    separator=' | ',
    empty_string='No coordinates',
    lng_first=False,
    num_digits=6
).add_to(mymap)

# Add markers for launch site and coastline
folium.Marker([launch_site_lat, launch_site_lon], popup="Launch Site", icon=folium.Icon(color='blue')).add_to(mymap)
folium.Marker([rail_lat, rail_lon], popup="Tri-Rail Opa-locka Station ", icon=folium.Icon(color='green')).add_to(mymap)

# Add a marker showing distance to coastline using DivIcon
distance_marker = folium.Marker(
    location=[rail_lat, rail_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
    )
)
distance_marker.add_to(mymap)

folium.PolyLine([(launch_site_lat, launch_site_lon), (rail_lat, rail_lon)], color='red').add_to(mymap)

# display map
mymap


The closest major highway to 28.5733, -80.6513 (Kennedy Space Center) is Florida State Road 405 (FL-405), also known as Columbia Boulevard. It connects to Interstate 95 (I-95), which is a major north-south highway running along the U.S. East Coast.

In [98]:
# launch site coordinates 
launch_site_lat = 28.5733
launch_site_lon = -80.6513

# Record of closest Highway (Columbia Boulevard) 
highway_lat =  28.5277
highway_lon = -80.7992

# Calculate the distance between the launch site and the coastline
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

# Create a Folium map centered on the launch site
mymap = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Enable MousePosition plugin to track coordinates
MousePosition(
    position='topright',
    separator=' | ',
    empty_string='No coordinates',
    lng_first=False,
    num_digits=6
).add_to(mymap)

# Add markers for launch site and coastline
folium.Marker([launch_site_lat, launch_site_lon], popup="Launch Site", icon=folium.Icon(color='blue')).add_to(mymap)
folium.Marker([highway_lat, highway_lon], popup="Tri-Rail Opa-locka Station ", icon=folium.Icon(color='green')).add_to(mymap)

# Add a marker showing distance to coastline using DivIcon
distance_marker = folium.Marker(
    location=[highway_lat, highway_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
    )
)
distance_marker.add_to(mymap)

folium.PolyLine([(launch_site_lat, launch_site_lon), (highway_lat, highway_lon)], color='red').add_to(mymap)

# display map
mymap


1. **Railway** findings (299.91 Km): launch sites are generally far from railways. That's because railways often support heavy cargo transport. Placing railways near launch pads would introduce safety hazards if a train were nearby during a rocket launch.

2. **Highways** findings (15.31 Km): Highways provide easy access for transporting rockets, fuel, and other critical supplies. However, launch sites aren’t directly on highways for safety reasons. These launches geneerate intense vibrations and sonic booms, which could affect nearby infrastructures.

3. **City** findings (15.87 Km): Similar to highways, a reasonable distance ensures that sound waves, vibrations, and potential failures don't affect urban areas. Rockets produce extreme noise levels, which can damage buildings and disrupt daily life.

4. **Coastlines** findings (7.86 Km): the launch pad is very close the coastlines. The reason behind this is that launching rockets near coastlines minimizes risks if something goes wrong - the risk of collisions with aircrafts in populated areas. As a result, if a launch fails, the debris falls into the water.

# Vandenberg Space Launch Complex

Let's try and affirm the geographical patterns. I will do the same work, this time, with the Vandenberg Space Launch Complex (VAFB SLC-4E), based in California.


1. Closest Coastline:
Location: Near Point Conception, California
Coordinates: 34.4481° N, -120.4610° W

3. Closest Highway:
Location: California State Route 1 (CA-1), near Lompoc, CA
Coordinates: 34.6389° N, -120.4736° W

3. Closest Railway:
Location: Lompoc, CA (nearest railway infrastructure)
Coordinates: 34.7483° N, -120.5182° W

4. Closest City:
Location: Lompoc, California
Coordinates: 34.6327° N, -120.6134° W

In [17]:
# VAFB SLC-4E coordinates 
launch_site_lat2 = 34.632834
launch_site_lon2 = -120.610745

# Record of the closest coastline coordinates 
coastline_lat2 = 34.4481
coastline_lon2 = -120.4610

# Calculate the distance between the launch site and the coastline
distance_coastline2 = calculate_distance(launch_site_lat2, launch_site_lon2, coastline_lat2, coastline_lon2)

# Create a Folium map centered on the launch site
mymap = folium.Map(location=[launch_site_lat2, launch_site_lon2], zoom_start=10)

# Enable MousePosition plugin to track coordinates
MousePosition(
    position='topright',
    separator=' | ',
    empty_string='No coordinates',
    lng_first=False,
    num_digits=6
).add_to(mymap)

# Add markers for launch site and coastline
folium.Marker([launch_site_lat2, launch_site_lon2], popup="Launch Site", icon=folium.Icon(color='blue')).add_to(mymap)
folium.Marker([coastline_lat2, coastline_lon2], popup="Closest Coastline", icon=folium.Icon(color='green')).add_to(mymap)

# Add a marker showing distance to coastline using DivIcon
distance_marker = folium.Marker(
    location=[coastline_lat2, coastline_lon2],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline2),
    )
)
distance_marker.add_to(mymap)

folium.PolyLine([(launch_site_lat2, launch_site_lon2), (coastline_lat2, coastline_lon2)], color='red', weight=1).add_to(mymap)

# display map
mymap


In [101]:
# VAFB SLC-4E coordinates 
launch_site_lat2 = 34.632834
launch_site_lon2 = -120.610745

# Record of the closest highway coordinates 
highway_lat2 = 34.6389
highway_lon2 = -120.4736

# Calculate the distance between the launch site and the Highway
distance_highway2 = calculate_distance(launch_site_lat2, launch_site_lon2, highway_lat2, highway_lon2)

# Create a Folium map centered on the launch site
mymap = folium.Map(location=[launch_site_lat2, launch_site_lon2], zoom_start=10)

# Enable MousePosition plugin to track coordinates
MousePosition(
    position='topright',
    separator=' | ',
    empty_string='No coordinates',
    lng_first=False,
    num_digits=6
).add_to(mymap)

# Add markers for launch site and Highway
folium.Marker([launch_site_lat2, launch_site_lon2], popup="Launch Site", icon=folium.Icon(color='blue')).add_to(mymap)
folium.Marker([highway_lat2, highway_lon2], popup="Closest Highway", icon=folium.Icon(color='green')).add_to(mymap)

# Add a marker showing distance using DivIcon
distance_marker = folium.Marker(
    location=[coastline_lat2, coastline_lon2],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway2),
    )
)
distance_marker.add_to(mymap)

folium.PolyLine([(launch_site_lat2, launch_site_lon2), (highway_lat2, highway_lon2)], color='red').add_to(mymap)

# display map
mymap


In [102]:
# VAFB SLC-4E coordinates 
launch_site_lat2 = 34.632834
launch_site_lon2 = -120.610745

# Record of the closest railway coordinates 
rail_lat2 = 34.7483  
rail_lon2 = -120.5182

# Calculate the distance between the launch site and the railway
distance_rail2 = calculate_distance(launch_site_lat2, launch_site_lon2, rail_lat2, rail_lon2)

# Create a Folium map centered on the launch site
mymap = folium.Map(location=[launch_site_lat2, launch_site_lon2], zoom_start=10)

# Enable MousePosition plugin to track coordinates
MousePosition(
    position='topright',
    separator=' | ',
    empty_string='No coordinates',
    lng_first=False,
    num_digits=6
).add_to(mymap)

# Add markers for launch site and the railway
folium.Marker([launch_site_lat2, launch_site_lon2], popup="Launch Site", icon=folium.Icon(color='blue')).add_to(mymap)
folium.Marker([rail_lat2, rail_lon2], popup="Closest Railway", icon=folium.Icon(color='green')).add_to(mymap)

# Add a marker showing distance using DivIcon
distance_marker = folium.Marker(
    location=[rail_lat2, rail_lon2],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_rail2),
    )
)
distance_marker.add_to(mymap)

folium.PolyLine([(launch_site_lat2, launch_site_lon2), (rail_lat2, rail_lon2)], color='red', weight=1).add_to(mymap)

# display map
mymap


In [103]:
# VAFB SLC-4E coordinates 
launch_site_lat2 = 34.632834
launch_site_lon2 = -120.610745

# Record of the closest city coordinates 
city_lat2 = 34.6327 
city_lon2 = -120.6134

# Calculate the distance between the launch site and the city
distance_city2 = calculate_distance(launch_site_lat2, launch_site_lon2, city_lat2, city_lon2)

# Create a Folium map centered on the launch site
mymap = folium.Map(location=[launch_site_lat2, launch_site_lon2], zoom_start=10)

# Enable MousePosition plugin to track coordinates
MousePosition(
    position='topright',
    separator=' | ',
    empty_string='No coordinates',
    lng_first=False,
    num_digits=6
).add_to(mymap)

# Add markers for launch site and city
folium.Marker([launch_site_lat2, launch_site_lon2], popup="Launch Site", icon=folium.Icon(color='blue')).add_to(mymap)
folium.Marker([city_lat2, city_lon2], popup="Closest City", icon=folium.Icon(color='green')).add_to(mymap)

# Add a marker showing distance using DivIcon
distance_marker = folium.Marker(
    location=[city_lat2, city_lon2],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline2),
    )
)
distance_marker.add_to(mymap)

folium.PolyLine([(launch_site_lat2, launch_site_lon2), (city_lat2, city_lon2)], color='red', weight=1).add_to(mymap)

# display map
mymap


Findings summary for the Vandenberg Space Launch Complex:

1. **Closest railway**: 15.38 Km
2. **Closest highway** : 12.57 Km
3. **Closest city** : 24.71 Km
4. **Closest Coastline**: 24.71 Km

The Vandenberg Space Launch Complex distance results show a **different pattern** compared to Kennedy Space Center (Florida).

### Difference in Pattern Explained 

| Feature            | Vandenberg (CA) | Kennedy (FL) | Explanation |
|--------------------|----------------|--------------|-----------|
| **Coastline Distance** | 24.71 km | 7.86 km | Kennedy benefits from ocean launches |
| **City Distance** | 24.71 km | 15.8 km | Vandenberg is more isolated for security |
| **Railway Distance** | 15.38 km | 299.91 km | Kennedy relies on sea transport, Vandenberg uses rail |
| **Highway Distance** | 12.57 km | 15.31 km | Both need highways, but Vandenberg is slightly closer |

### Final Thought!

1. I notice that Vandenberg is further from the coastline – Unlike Kennedy Space Center, which launches over the ocean, Vandenberg sends rockets into polar orbits, so proximity to the coastline is less critical.
2.  Kennedy Space Center is farther from railways – The massive 299.91 km gap indicates Florida's reliance on highway & sea transport instead of railways.
3. Vandenberg has more buffer distance from cities – Its 24.71 km separation suggests an extra safety margin compared to Kennedy.

# Project by:

  [Carl Legros](https://www.linkedin.com/in/carllegros/)